In [1]:
import pandas as pd
import numpy as np

df = pd.read_excel (r'C:\Users\swapnil\ML_learning\iste_ml\learning_resources\logistic_regression_iris.xlsx')
df=df.dropna()
df.reset_index(drop=True)
df

,sepal_length,sepal_width2,petal_length,petal_width,class
0,6.3,2.9,4.3,1.3,1
1,6.3,2.3,4.0,1.3,1
2,6.3,3.4,5.6,2.4,0
3,6.3,2.5,3.0,1.1,1
4,6.3,2.3,4.4,1.3,1
...,...,...,...,...,...
94,6.3,2.8,4.9,2.0,0
95,6.3,2.9,4.7,1.4,1
96,6.3,3.0,5.2,2.0,0
97,6.3,3.1,5.5,1.8,0


In [84]:
X=df.iloc[:,:-1].values
norm_X=np.linalg.norm(X)
X=X/norm_X
Y=df.iloc[:,-1].values
Y.shape=(99,1)
print(X.shape,Y.shape)
X_Train,X_Test=X[:60,:],X[60:,:]
Y_Train,Y_Test=Y[:60,:],Y[60:,]
X_Train,X_Test,Y_Train,Y_Test=X_Train.T,X_Test.T,Y_Train.T,Y_Test.T
print(X_Train.shape,X_Test.shape,Y_Train.shape,Y_Test.shape)
X_Train,X_Test,Y_Train,Y_Test

(99, 4) (99, 1)
(4, 60) (4, 39) (1, 60) (1, 39)


(array([[0.07273973, 0.07273973, 0.07273973, 0.07273973, 0.07273973,
         0.07273973, 0.07273973, 0.07273973, 0.07273973, 0.07273973,
         0.07273973, 0.07273973, 0.07273973, 0.07273973, 0.07273973,
         0.07273973, 0.07273973, 0.07273973, 0.07273973, 0.07273973,
         0.07273973, 0.07273973, 0.07273973, 0.07273973, 0.07273973,
         0.07273973, 0.07273973, 0.07273973, 0.07273973, 0.07273973,
         0.07273973, 0.07273973, 0.07273973, 0.07273973, 0.07273973,
         0.07273973, 0.07273973, 0.07273973, 0.07273973, 0.07273973,
         0.07273973, 0.07273973, 0.07273973, 0.07273973, 0.07273973,
         0.07273973, 0.07273973, 0.07273973, 0.07273973, 0.07273973,
         0.07273973, 0.07273973, 0.07273973, 0.07273973, 0.07273973,
         0.07273973, 0.07273973, 0.07273973, 0.07273973, 0.07273973],
        [0.03348337, 0.02655578, 0.03925636, 0.02886497, 0.02655578,
         0.03232877, 0.04387476, 0.03694717, 0.03001957, 0.02886497,
         0.03463797, 0.03463797, 

In [85]:
def sigmoid(X):
    return (1/(1+np.exp(-X)))

def layers(X,Y):
    n_X=X.shape[0]
    n_h=5
    n_Y=Y.shape[0]
    return (n_X,n_h,n_Y)

def start_parameters(n_X,n_h,n_Y):
    W1=np.random.randn(n_h,n_X)*0.01
    b1=np.zeros((n_h,1))
    W2=np.random.randn(n_Y,n_h)*0.01
    b2=np.zeros((n_Y,1))
    
    parameters={
        "W1": W1,
        "b1": b1,
        "W2": W2,
        "b2": b2
    }
    
    return parameters

def forward_prop(X,parameters):
    
    W1=parameters["W1"]
    b1=parameters["b1"]
    W2=parameters["W2"]
    b2=parameters["b2"]
    
    Z1=np.dot(W1,X)+b1
    A1=np.tanh(Z1)
    Z2=np.dot(W2,Z1)+b2
    A2=sigmoid(Z2)
    
    sec_parameters={
        "Z1": Z1,
        "A1": A1,
        "Z2": Z2,
        "A2": A2
    }
    
    return sec_parameters

def find_cost(A2,Y):
    m=Y.shape[1]
    cost=-np.sum(np.multiply(np.log(A2),Y)+np.multiply(np.log(1-A2),(1-Y)))/m
    return cost

def backward_prop(parameters,sec_parameters,X,Y):
    m=X.shape[1]
    
    W1=parameters["W1"]
    W2=parameters["W2"]
    A1=sec_parameters["A1"]
    A2=sec_parameters["A2"]
    
    dZ2=A2-Y
    dW2=np.dot(dZ2,A1.T)/m
    db2=np.sum(dZ2,axis=1,keepdims=True)/m
    dZ1=np.dot(W2.T,dZ2)*(1-np.power(A1,2))
    dW1=np.dot(dZ1,X.T)/m
    db1=np.sum(dZ1,axis=1,keepdims=True)/m
    
    tri_parameters={
        "dW1": dW1,
        "dW2": dW2,
        "db1": db1,
        "db2": db2
    }
    
    return tri_parameters

def update_parameters(parameters,tri_parameters,learning_rate):
    W1 = parameters["W1"]
    b1 = parameters["b1"]
    W2 = parameters["W2"]
    b2 = parameters["b2"]
    
    dW1 = tri_parameters["dW1"]
    db1 = tri_parameters["db1"]
    dW2 = tri_parameters["dW2"]
    db2 = tri_parameters["db2"]
    
    W1 = W1-learning_rate*dW1
    b1 = b1-learning_rate*db1
    W2 = W2-learning_rate*dW2
    b2 = b2-learning_rate*db2
    
    parameters = {"W1": W1,
                  "b1": b1,
                  "W2": W2,
                  "b2": b2}
    
    return parameters


In [86]:
def lg_model(X,Y,num_iterations):
    n_X,n_h,n_Y=layers(X,Y)
    parameters=start_parameters(n_X,n_h,n_Y)
    
    for i in range(0, num_iterations):
        sec_parameters=forward_prop(X,parameters)
        cost=find_cost(sec_parameters["A2"],Y)
        tri_parameters = backward_prop(parameters,sec_parameters,X,Y)
        parameters = update_parameters(parameters,tri_parameters,0.01)
        if i%100000==0:
            print(cost)
    return parameters

In [87]:
def lg_prediction(parameters,X):
    train_prediction = forward_prop(X,parameters)
    return ((train_prediction["A2"]>0.5)*1)

In [88]:
def logistic_regression(X_Train,X_Test,Y_Train,Y_Test):
    parameters=lg_model(X_Train,Y_Train,1000000)
    train_prediction=lg_prediction(parameters,X_Train)
    train_accuracy=99-np.sum(np.abs(train_prediction-Y_Train))
    train_accuracy = (100*train_accuracy)/99
    test_prediction=lg_prediction(parameters,X_Test)
    test_accuracy=99-np.sum(np.abs(test_prediction-Y_Test))
    test_accuracy=(100*test_accuracy)/99
    return train_accuracy,test_accuracy


In [89]:
train_accuracy,test_accuracy=logistic_regression(X_Train,X_Test,Y_Train,Y_Test)


0.6931485697078859
0.6754151152512055
0.20296002805406832
0.11586089718286557
0.09112360762485529
0.07789584037934534
0.06918608860293443
0.06269223232792544
0.057440283471607755
0.05296725715376974


In [90]:
print(train_accuracy)
print(test_accuracy)

98.98989898989899
95.95959595959596
